## R-alpha

In [ ]:
!pip install smartapi-python
!pip install websocket-client
!pip install logzero
!pip install pyotp
!pip install requests
!pip install numpy
!pip install pandas
!pip install yfinance

In [ ]:
# package import statement
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
from connect_api import connect_api, create_api_session
from data import get_data_stream, token_lookup, get_instrument_list, hist_data, hist_data_from_now
from strategy import expected_buy_price, expected_sell_price, trend_detection, exit_long, exit_short
from order import place_limit_order, place_market_order
from momentum import get_top_nse_stocks_by_volume, momentum_strategy
import json
import threading
import time   

In [ ]:
api_key = '2VTNvx7z'
username = 'R258447'
pwd = '1499'
token_id = '3URP5C2ZUQSYTSEFELMM2CFJGM'

In [ ]:
smartApi = connect_api(api_key)
data = create_api_session(token_id, username, pwd, smartApi)

### Get Ticker Data

In [ ]:
instrument_list = get_instrument_list()

ticker_list = ["ITC"]
token_list_0 = []

for ticker in ticker_list:
    token = token_lookup(ticker, instrument_list, exchange="NSE")
    token_list_0.append(token)

token_list = [
        {
            "exchangeType": 1,
            "tokens": token_list_0
        }
    ]


print(token_list)

## Create Live Data Stream

In [ ]:
correlation_id = "stream_2" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 1 #1 for LTP, 2 for Quote and 3 for SnapQuote
    
sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

## Momentum Strategy

In [ ]:
top_10_stocks = get_top_nse_stocks_by_volume(instrument_list[:100])

In [ ]:
top_10_stocks

In [75]:
import pandas as pd

ticker_ltp = {}
position = 0
momentum_short_window = 3
momentum_long_window = 12
trades = {}
capital = 1500

def momentum_strategy(tick):

    global position
    global trades
    global capital
    global ticker_ltp
    ### add ltp and return to dictionary
    token = tick['token']
    ltp = tick['last_traded_price']/100

    print(ltp)

    momentum_signal = 0

    if token in ticker_ltp:
        ticker_ltp[token]['ltp'].append(ltp)
        momentum_short = pd.Series(ticker_ltp[token]['ltp']).rolling(window=momentum_short_window).mean().iloc[-1]

        ticker_ltp[token]['momentum_short'].append(momentum_short)
        momentum_long = pd.Series(ticker_ltp[token]['ltp']).rolling(window=momentum_long_window).mean().iloc[-1]
        ticker_ltp[token]['momentum_long'].append(momentum_long)
    else:
        ticker_ltp[token] = {
            'ltp': [ltp],
            'momentum_short':[None],
            'momentum_long': [None]
        }


    # Check momentum signal
    if (ticker_ltp[token]['momentum_short'][-1] is not None) and (ticker_ltp[token]['momentum_long'][-1] is not None):
        if ticker_ltp[token]['momentum_short'][-1] > ticker_ltp[token]['momentum_long'][-1]:
            momentum_signal = 1
        elif ticker_ltp[token]['momentum_short'][-1] < ticker_ltp[token]['momentum_long'][-1]:
            momentum_signal = -1

    #create trade

    if momentum_signal == 1 and position == 0:
        quantity = capital // ltp
        price = ltp
        position = 1
        capital = 0

        if token in trades:
            trades[token]['buy_prices'].append(price)
            trades[token]['quantity'].append(quantity)
            # trades[token]['sell_price'].append(None)
        
        else:
            trades[token] = {
            'buy_prices': [price],
            'quantity': [quantity],
            'sell_price': []
            }

        place_limit_order(instrument_list,"ITC",'BUY',price,quantity,smartApi, exchange="NSE")

    elif momentum_signal == -1 and position == 1:
        quantity = trades[token]['quantity'][-1]
        price = ltp
        position = 0

        capital = quantity*price
        if token in trades:
            trades[token]['sell_price'].append(price)
        
        else:
            trades[token] = {
            'sell_price':price
            }

        place_limit_order(instrument_list,"ITC",'SELL',price,quantity,smartApi, exchange="NSE")

def on_data(ws, message):
    momentum_strategy(message)

sws.on_data = on_data

In [76]:
sws.connect()

[I 240507 08:42:44 data:94] on open


440.5
440.5
440.5
440.5
440.5
440.5
440.55
440.55
440.6
440.5
440.45
440.55
440.6
440.45
440.6
440.55
440.55
440.5
440.45
440.55
440.55
440.55
440.6
440.6
440.6
440.55
440.6
440.6
440.6
440.6
440.6
440.6
440.55
440.6
440.6
440.6
440.6
440.6
440.6
440.5
440.5
440.5
440.5
440.6
440.6
440.6


[E 240507 08:43:19 smartConnect:243] Error occurred while making a POST request to https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder. Error: INVALID PARAMETERS. URL: https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': '3e:4c:b1:f5:1f:1c', 'Accept': 'application/json', 'X-PrivateKey': '2VTNvx7z', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'variety': 'NORMAL', 'tradingsymbol': 'ITC-EQ', 'symboltoken': '1660', 'transactiontype': 'BUY', 'exchange': 'NSE', 'ordertype': 'LIMIT', 'producttype': 'INTRADAY', 'duration': 'DAY', 'price': 440.6, 'quantity': 0.0}, Response: {'status': False, 'message': 'INVALID PARAMETERS', 'errorcode': 'AB1007', 'data': {'script': 'ITC-EQ', 'orderid': None, 'uniqueorderid': '388e169b-ca97-4295-a3ba-6a4cdd11a8be'}}
[E 240507 08:43:19 smartConnect:336] API request f

440.6
440.6
440.6
440.6
440.6
440.45


[E 240507 08:43:23 smartConnect:243] Error occurred while making a POST request to https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder. Error: INVALID PARAMETERS. URL: https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': '3e:4c:b1:f5:1f:1c', 'Accept': 'application/json', 'X-PrivateKey': '2VTNvx7z', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'variety': 'NORMAL', 'tradingsymbol': 'ITC-EQ', 'symboltoken': '1660', 'transactiontype': 'SELL', 'exchange': 'NSE', 'ordertype': 'LIMIT', 'producttype': 'INTRADAY', 'duration': 'DAY', 'price': 440.45, 'quantity': 0.0}, Response: {'status': False, 'message': 'INVALID PARAMETERS', 'errorcode': 'AB1007', 'data': {'script': 'ITC-EQ', 'orderid': None, 'uniqueorderid': 'cc668f28-d8b7-4ec1-a2a1-66abdd665ea0'}}
[E 240507 08:43:23 smartConnect:336] API request

...

439.85


[E 240507 09:05:53 smartConnect:243] Error occurred while making a POST request to https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder. Error: INVALID PARAMETERS. URL: https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': '3e:4c:b1:f5:1f:1c', 'Accept': 'application/json', 'X-PrivateKey': '2VTNvx7z', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'variety': 'NORMAL', 'tradingsymbol': 'ITC-EQ', 'symboltoken': '1660', 'transactiontype': 'SELL', 'exchange': 'NSE', 'ordertype': 'LIMIT', 'producttype': 'INTRADAY', 'duration': 'DAY', 'price': 439.85, 'quantity': 0.0}, Response: {'status': False, 'message': 'INVALID PARAMETERS', 'errorcode': 'AB1007', 'data': {'script': 'ITC-EQ', 'orderid': None, 'uniqueorderid': 'e730507d-c0d1-40e9-8b84-8898e0fedabf'}}
[E 240507 09:05:53 smartConnect:336] API request

439.9
439.85
439.9
439.95


[E 240507 09:05:57 smartConnect:243] Error occurred while making a POST request to https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder. Error: INVALID PARAMETERS. URL: https://apiconnect.angelbroking.com/rest/secure/angelbroking/order/v1/placeOrder, Headers: {'Content-type': 'application/json', 'X-ClientLocalIP': '127.0.0.1', 'X-ClientPublicIP': '106.193.147.98', 'X-MACAddress': '3e:4c:b1:f5:1f:1c', 'Accept': 'application/json', 'X-PrivateKey': '2VTNvx7z', 'X-UserType': 'USER', 'X-SourceID': 'WEB'}, Request: {'variety': 'NORMAL', 'tradingsymbol': 'ITC-EQ', 'symboltoken': '1660', 'transactiontype': 'BUY', 'exchange': 'NSE', 'ordertype': 'LIMIT', 'producttype': 'INTRADAY', 'duration': 'DAY', 'price': 439.95, 'quantity': 0.0}, Response: {'status': False, 'message': 'INVALID PARAMETERS', 'errorcode': 'AB1007', 'data': {'script': 'ITC-EQ', 'orderid': None, 'uniqueorderid': 'dc3a389c-e970-4747-863e-0714a8fb31e7'}}
[E 240507 09:05:57 smartConnect:336] API request 

439.95
439.95
439.95
439.9
439.95
439.95


In [ ]:
trades

## misc

In [ ]:
correlation_id = "stream_1"
action = 1
mode = 1
token_list = [
        {
            "exchangeType": 1,
            "tokens": ["26009"]
        }
    ]

sws = get_data_stream(token_id, api_key, username, smartApi.getfeedToken(), token_list, correlation_id, action, mode)

In [ ]:
# Create a new thread and pass the function to it
my_thread = threading.Thread(target=sws.connect())

# Start the thread
my_thread.start()
time.sleep(10)

In [ ]:
place_market_order(instrument_list,ticker="ITC",buy_sell="BUY",price=0,quantity=1,obj=smartApi, sl=0,sqof=0,exchange="NSE")

In [ ]:
nifty_50 = hist_data_from_now(smartApi, "NIFTYBEES", 2000, "ONE_DAY", instrument_list)

In [ ]:
nifty_50.to_csv("niftybees_2000day_1day.csv")

In [ ]:
instrument_list[0]